In [1]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
# Plotting tools

import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
## import csv
df1 = pd.read_csv("data_1.csv")
df2 = pd.read_csv("data_2.csv")
df3 = pd.read_csv("data_3.csv")


In [22]:
import os, glob
import pandas as pd



all_files = glob.glob(os.path.join("data_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv( "merged.csv")

In [15]:
from google.colab import files
files.download("merged.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
df_merged

,Index,Title,Article,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,0,Rs 400-cr ‘bungling’ by Amar Singh: HC seeks E...,The Allahabad High Court has directed the Enfo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,J&K HC orders FIR against teacher who fails to...,The J&K High Court on Friday asked a teacher t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Aashish Gupta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Payal Hathi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,P.S. Pasricha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
8584,2309,Is asteroid Apophis capable of triggering a ts...,It was around a few months back that Iain McDo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8585,2310,New 'dengue-like disease' spreading among chil...,The number of children affected by a new disea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8586,2311,"GameStop shares surge 1,700% in 2 weeks, Nasda...",As the unusual market surge of Gamestop shares...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8587,2312,"Over 4000 deaths in a single day, US chocks as...",It was during the summer peak of the coronavir...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
keep_col = ['Title','Article']
new_f = df_merged[keep_col]
new_f.to_csv("newFile.csv", index=False)

In [31]:
new_f

,Title,Article
0,Rs 400-cr ‘bungling’ by Amar Singh: HC seeks E...,The Allahabad High Court has directed the Enfo...
1,J&K HC orders FIR against teacher who fails to...,The J&K High Court on Friday asked a teacher t...
2,Aashish Gupta,NaN
3,Payal Hathi,NaN
4,P.S. Pasricha,NaN
...,...,...
8584,Is asteroid Apophis capable of triggering a ts...,It was around a few months back that Iain McDo...
8585,New 'dengue-like disease' spreading among chil...,The number of children affected by a new disea...
8586,"GameStop shares surge 1,700% in 2 weeks, Nasda...",As the unusual market surge of Gamestop shares...
8587,"Over 4000 deaths in a single day, US chocks as...",It was during the summer peak of the coronavir...


In [27]:
from google.colab import files
files.download("newFile.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
new_f=new_f.dropna(subset = ['Article'])
new_f=new_f.reset_index(drop=True)

In [39]:
new_f

,Title,Article
0,Rs 400-cr ‘bungling’ by Amar Singh: HC seeks E...,The Allahabad High Court has directed the Enfo...
1,J&K HC orders FIR against teacher who fails to...,The J&K High Court on Friday asked a teacher t...
2,Crime in Cyberia: an incomplete list of offenders,An unknown Indian hacker has been charged with...
3,450 cannabis plants found on govt land in Mundhwa,The interrogation of two marijuana peddlers ab...
4,"Nagaland: 9 bullet-riddled bodies found,Karbi-...",Nine highly decomposed bodies were found stack...
...,...,...
8576,Is asteroid Apophis capable of triggering a ts...,It was around a few months back that Iain McDo...
8577,New 'dengue-like disease' spreading among chil...,The number of children affected by a new disea...
8578,"GameStop shares surge 1,700% in 2 weeks, Nasda...",As the unusual market surge of Gamestop shares...
8579,"Over 4000 deaths in a single day, US chocks as...",It was during the summer peak of the coronavir...


In [40]:
count = new_f['Article'].str.split().str.len()
count.index = count.index.astype(str) + ' words:'
count.sort_index(inplace=True)
count

0 words:       341
1 words:       335
10 words:      814
100 words:     184
1000 words:    331
              ... 
995 words:     303
996 words:     245
997 words:     252
998 words:     343
999 words:     245
Name: Article, Length: 8581, dtype: int64

In [42]:
new_f['totalwords'] = [len(x.split()) for x in new_f['Article'].tolist()]

In [45]:
new_f

,Title,Article,totalwords
0,Rs 400-cr ‘bungling’ by Amar Singh: HC seeks E...,The Allahabad High Court has directed the Enfo...,341
1,J&K HC orders FIR against teacher who fails to...,The J&K High Court on Friday asked a teacher t...,335
2,Crime in Cyberia: an incomplete list of offenders,An unknown Indian hacker has been charged with...,382
3,450 cannabis plants found on govt land in Mundhwa,The interrogation of two marijuana peddlers ab...,372
4,"Nagaland: 9 bullet-riddled bodies found,Karbi-...",Nine highly decomposed bodies were found stack...,449
...,...,...,...
8576,Is asteroid Apophis capable of triggering a ts...,It was around a few months back that Iain McDo...,412
8577,New 'dengue-like disease' spreading among chil...,The number of children affected by a new disea...,457
8578,"GameStop shares surge 1,700% in 2 weeks, Nasda...",As the unusual market surge of Gamestop shares...,586
8579,"Over 4000 deaths in a single day, US chocks as...",It was during the summer peak of the coronavir...,327


In [44]:
new_f.describe()

,totalwords
count,8581.000000
mean,510.569048
std,442.347599
min,2.000000
25%,279.000000
50%,388.000000
75%,590.000000
max,5884.000000


In [49]:
new_f.drop(new_f[new_f.totalwords < 100].index, inplace=True)
new_f=new_f.reset_index(drop=True)

In [50]:
new_f

,Title,Article,totalwords
0,Rs 400-cr ‘bungling’ by Amar Singh: HC seeks E...,The Allahabad High Court has directed the Enfo...,341
1,J&K HC orders FIR against teacher who fails to...,The J&K High Court on Friday asked a teacher t...,335
2,Crime in Cyberia: an incomplete list of offenders,An unknown Indian hacker has been charged with...,382
3,450 cannabis plants found on govt land in Mundhwa,The interrogation of two marijuana peddlers ab...,372
4,"Nagaland: 9 bullet-riddled bodies found,Karbi-...",Nine highly decomposed bodies were found stack...,449
...,...,...,...
8417,Is asteroid Apophis capable of triggering a ts...,It was around a few months back that Iain McDo...,412
8418,New 'dengue-like disease' spreading among chil...,The number of children affected by a new disea...,457
8419,"GameStop shares surge 1,700% in 2 weeks, Nasda...",As the unusual market surge of Gamestop shares...,586
8420,"Over 4000 deaths in a single day, US chocks as...",It was during the summer peak of the coronavir...,327


In [51]:
new_f.describe()

,totalwords
count,8422.000000
mean,518.962123
std,442.209540
min,100.000000
25%,286.000000
50%,392.000000
75%,596.000000
max,5884.000000


In [53]:
new_f.to_csv("scrapped_final.csv", index=False)

In [54]:
from google.colab import files
files.download("scrapped_final.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>